# &#128214; Lab 6: Points-to Analysis

## &#128214; Exercise 1: Building Points-to Analysis

### &#127919; Objective
So far, our model could be used to perform **intraprocedural** static analysis, which is the examination of individual functions. 
However, real-world programs have function calls, where data and control flow traverse from one function to another. 
To capture a more accurate representation of a program's behavior, it's important to extend our analysis beyond the boundaries of single functions.

In this exercise, we aim to enhance our model to conduct **interprocedural** analysis, enabling it to follow the flow of data and control across function calls. 
This leap will not only provide a more comprehensive understanding of the program's behavior but will also set the foundation for advanced analysis techniques.

One of the key aspects of interprocedural analysis is the call graph construction.
However, to build a call graph, one needs a points-to set for given variables, i.e., we need to perform a points-to analysis. 
This analysis helps in determining the potential objects a variable can point to, which in turn aids in identifying the possible targets of function calls. 

---

### &#128214; Background

In program analysis, understanding how data flows and how control is propagated across different procedures is important. 
This gets particularly significant when dealing with object-oriented or procedural programming paradigms where data and control frequently cross function boundaries.

- **Intraprocedural Analysis:** This type of analysis is scoped within a single procedure or function. 
The control flow graph (CFG) you've built in previous exercises primarily supports intraprocedural analysis.

- **Interprocedural Analysis:** This expands the analysis scope to multiple procedures or functions. 
It helps in understanding how data and control flow across different parts of a program.

- **Points-to Analysis:** This is a technique used to determine the set of objects that a variable can point to.

- **Call Graph:** A call graph is a directed graph that represents calling relationships between functions in a program. Each node represents a function and each edge (A, B) indicates that function A calls function B.

---

### &#10145; Tasks
- Analyze a given piece of code to build a class hierarchy. This hierarchy should capture the parent-child relationships between classes.
- For a given class, find the highest parent in the hierarchy.
- From the highest parent, gather all subclasses (children, grandchildren, etc.) within the hierarchy.
- Build the points-to set using the class hierarchy

### Import the necessary library

&#128161; *In the following cell, we will import the library needed for this exercise:*
- `ast`: a module of the python standard library to transform Python code in its AST representation


In [ ]:
import ast

&#9874; Utility function

We provide a utility function called `print_dict` that can be used to print an indented representation of a `dict` object in Python.

In [ ]:
import json

def print_dict(dictionary):
    """
    Prints a dictionary in a beautified and indented format.

    Parameters:
    dictionary: The dictionary to be printed.
    """
    print(json.dumps(dictionary, indent=4))

Python code

&#128161; The following cell contains a string that represents the Python code that will be analyzed through this exercise

In [ ]:
code = """
class Language:
    def __init__(self):
        pass
    def say_hello(self, first_name):
        print(f"Saluton {first_name} el ĝenerala lingvo.")

class French(Language):
    def say_hello(self, first_name):
        print(f"Bonjour {first_name}.")

class English(Language):
    def say_hello(self, first_name):
        print(f"Hello {first_name}.")

class German(Language):
    def say_hello(self, first_name):
        print(f"Hallo {first_name}.")

class Spanish(Language):
    def say_hello(self, first_name):
        print(f"Hola {first_name}.")

class Italian(Language):
    def say_hello(self, first_name):
        print(f"Ciao {first_name}.")

class Portuguese(Language):
    def say_hello(self, first_name):
        print(f"Olá {first_name}.")

if x > 5:
    obj = French()
else:
    obj = English()
    
obj.say_hello("Patrick")
    
if y < 3:
    obj = German()
else:
    obj = Spanish()

obj.say_hello("John")
"""


### &#128161; Helper Function Explanation

In Python, both class instantiation and function calling use the same syntax which leads to an ambiguity when analyzing the AST of a code snippet. 
Specifically, both operations appear as `ast.Call` nodes within the AST, which makes it difficult to distinguish between the two just by examining the syntax.

For instance, consider the following Python code snippet:

```python
class MyClass:
    pass

def my_function():
    pass

obj = MyClass()  # Class instantiation
my_function()    # Function call
```

In the AST, both `MyClass()` and `my_function()` will be represented as `ast.Call` nodes, which could lead to misinterpretation during analysis.

To address this issue and provide clearer analysis, we present a helper function that traverses the AST to identify and categorize all class and function definitions. 
This will later help in accurately interpreting `ast.Call` nodes.

#### Class and Function Analyzer

We define a class `ClassFunctionAnalyzer` which inherits from `ast.NodeVisitor`. 
This class overrides three methods: `visit_ClassDef`, `visit_FunctionDef`, and `visit_Call`.

```python
class ClassFunctionAnalyzer(ast.NodeVisitor):
    """
    A node visitor class that walks through the Abstract Syntax Tree (AST) to identify and record all class and function definitions within the code.

    Attributes:
    classes: A set to store unique class names encountered in the AST.
    functions: A set to store unique function names encountered in the AST.
    """
    def __init__(self):
        self.classes = set()
        self.functions = set()

    def visit_ClassDef(self, node):
        """
        Visits a class definition node, adding the class name to the classes set.

        Parameters:
        node: The class definition node being visited.
        """
        self.classes.add(node.name)
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        """
        Visits a function definition node, adding the function name to the functions set.

        Parameters:
        node: The function definition node being visited.
        """
        self.functions.add(node.name)
        self.generic_visit(node)
        
    def get_call_type(self, name):
        """
        Determines whether a given name corresponds to a class instantiation or a function call.

        Parameters:
        name: The name of the function or class.

        Returns:
        str: "class" if the name corresponds to a class, "function" if it corresponds to a function, or "unknown" otherwise.
        """
        if name in self.classes:
            return "class"
        elif name in self.functions:
            return "function"
        else:
            return "unknown"
```

#### Explanations:

- `__init__`: Initializes two sets, `classes` and `functions`, to store the names of classes and functions respectively.
  
- `visit_ClassDef`: This method is invoked for every `ast.ClassDef` node encountered. It adds the name of the class to the `classes` set.

- `visit_FunctionDef`: This method is invoked for every `ast.FunctionDef` node encountered. It adds the name of the function to the `functions` set.

- `visit_Call`: This method is invoked for every `ast.Call` node encountered. It checks whether the called entity is a class or a function by comparing the identifier against the collected sets of class and function names.

By using this helper function, you will be able to determine whether an `ast.Call` node represents a class instantiation or a function call, which is important for the points-to analysis you will be performing.
Of course, this solution is not optimal/perfect since we might have different functions with the same name in a program, but for simplicity we make the hypothesis that functions are unique. 

In [ ]:
class ClassFunctionAnalyzer(ast.NodeVisitor):
    """
    A node visitor class that walks through the Abstract Syntax Tree (AST) to identify and record all class and function definitions within the code.

    Attributes:
    classes: A set to store unique class names encountered in the AST.
    functions: A set to store unique function names encountered in the AST.
    """
    def __init__(self):
        self.classes = set()
        self.functions = set()

    def visit_ClassDef(self, node):
        """
        Visits a class definition node, adding the class name to the classes set.

        Parameters:
        node: The class definition node being visited.
        """
        self.classes.add(node.name)
        self.generic_visit(node)

    def visit_FunctionDef(self, node):
        """
        Visits a function definition node, adding the function name to the functions set.

        Parameters:
        node: The function definition node being visited.
        """
        self.functions.add(node.name)
        self.generic_visit(node)
        
    def get_call_type(self, name):
        """
        Determines whether a given name corresponds to a class instantiation or a function call.

        Parameters:
        name: The name of the function or class.

        Returns:
        str: "class" if the name corresponds to a class, "function" if it corresponds to a function, or "unknown" otherwise.
        """
        if name in self.classes:
            return "class"
        elif name in self.functions:
            return "function"
        else:
            return "unknown"

### &#128161; How to use the analyzer class to use throughout this lab

In [ ]:
tree = ast.parse(code)
analyzer = ClassFunctionAnalyzer()
analyzer.visit(tree)

&#128161; Ok, now we are able to make the difference between a function call and a call to a constructor by using the `get_call_type` on the analyzer object, let us continue.

In this exercise, the focus is on understanding and handling scenarios where a method call is made on an object, and the specific class of the object is unknown. 
This situation presents a challenge in determining the behavior of the method call. 
A simplistic yet effective approach to overcome this challenge is to consider the broadest over-approximation. 
This over-approximation assumes that the method call could potentially be linked to any class within a defined hierarchy. 
Therefore, a prerequisite task for this exercise is to build the class hierarchy (see [1]) from the given code.
Once the hierarchy is established, the objective is to identify method calls on objects with classes, and list all possible classes within the hierarchy that the method could belong to. 
This exercise serves as a precursor to developing a Points-to analysis. 
Through this task, you will grasp the concept of over-approximation, understand its impact on Points-to analysis, and lay the foundation for more precise analysis techniques to be explored in subsequent exercises.

[1] Dean, Jeffrey, David Grove, and Craig Chambers. "Optimization of object-oriented programs using static class hierarchy analysis." ECOOP’95—Object-Oriented Programming, 9th European Conference, Åarhus, Denmark, August 7–11, 1995 9. Springer Berlin Heidelberg, 1995.

### &#128161; Class Hierarchy

In the following cell, you will implement the `ClassHierarchyBuilder` class that extends the `ast.NodeVisitor` class to build the class hierarchy of the program under analysis.
Implement the `visit_ClassDef` to visit class definitions and build the class hierarchy.
If you need more methods, add some.


In [ ]:
class ClassHierarchyBuilder(ast.NodeVisitor):
    """
    A node visitor class to build a hierarchy of classes by traversing the Abstract Syntax Tree (AST).
    This class identifies parent-child relationships among classes and provides methods to analyze
    the resulting class hierarchy.

    Attributes:
    class_hierarchy: A dictionary representing the class hierarchy. Keys are class names,
                            and values are lists of base class names.
    """

    def __init__(self):
        self.class_hierarchy = {}

    def visit_ClassDef(self, node):
        """
        Visits a class definition node, extracting the class name and the names of its base classes,
        and updating the class hierarchy accordingly.

        Parameters:
        node: The class definition node being visited.
        """

### &#128161; In the following cell, you will test your hierarchy and print all the classes of the hierarchy that contains the `Spanish` class.

### &#128161; In the following cell, you will implement the construction of the points-to set using the class hierarchy.

Implement the `build_points_to_set` function that takes a node as a parameter.
This idea is to find all class instantiations in the program and for the class being instantiated: get the hierarchy and build the points-to set of the variable receiving the instantiation (i.e., the object).
For instance, an object's points-to analysis could be a set of class names.



In [ ]:
def build_points_to_set(node):
    """
    Analyzes an AST to gather information about potential class instantiations.
    
    Parameters:
    node: The root node of the AST.
    analyzer (ClassFunctionAnalyzer): An instance of ClassFunctionAnalyzer.
    
    Returns:
    dict: The points-to set.
    """

### &#128161; In the following cell, you will test your implementation.
Build the points-to set using the `build_points_to_set` function and print it using the `print_dict` function.

### &#10067; Questions

Explain the concept of over-approximation in the context of analyzing method calls on objects with unknown classes. Why is it useful, and what are its limitations?

Describe the process of building a class hierarchy from a given piece of code. Why is this hierarchy important for performing a broad over-approximation when analyzing method calls?

Did you identify any limitation to this technique? Explain.

## &#128214; Exercise 2: Refining Points-to Analysis

### &#127919; Objective
In the previous exercise, we ventured into interprocedural analysis and took our first steps towards understanding the essence of Points-to Analysis by dealing with class hierarchies. 
However, the approach was quite broad, focusing on the entire class hierarchy irrespective of whether a class was instantiated or not. 
In this exercise, we aim to refine the Points-to Analysis by sharpening in on the actual class instantiations within the code, and transitioning from a flow-insensitive to a flow-sensitive analysis.

---

### &#128214; Background

Precision in program analysis is important to understand the behavior of code and to minimize the chances of false positives or negatives. 
Two key aspects that can significantly enhance the precision of our Points-to Analysis are:

- **Considering Actual Class Instantiations:** Focusing on the classes that are actually instantiated in the code, as opposed to all classes in the hierarchy, can provide a more accurate picture of the possible objects a pointer can point to.

- **Flow Sensitivity:** Unlike flow-insensitive analysis, which disregards the order in which statements are executed, flow-sensitive analysis takes into account the sequence of statement execution.

### &#10145; Tasks
1. **Refinement by Class Instantiation:**
   - Modify your Points-to Analysis to only consider classes that are actually instantiated in the code.
   - Analyze the given piece of code and update your Points-to set based on actual class instantiations.

2. **Transition to Flow-sensitive Analysis:**
   - Adapt your Points-to Analysis to be flow-sensitive.
   - Implement an algorithm that considers the order of statement execution to build a more precise Points-to set.

By doing these refinements, you will enhance the precision of your Points-to Analysis, moving closer to a more precise representation of the program's behavior across different procedural contexts. 
This exercise will further build on your understanding of interprocedural analysis, preparing you for more advanced analysis techniques.

### In the following cell, you will implement the construction of the points-to set using only the classes instantiated.
Modify the `build_points_to_set` function to only retain classes that have been instantiated to decrease the lefvel of over-approximation in your analysis.

In [ ]:
def build_points_to_set(node):
    """
    Analyzes an AST to gather information about potential class instantiations.
    
    Parameters:
    node: The root node of the AST.
    analyzer (ClassFunctionAnalyzer): An instance of ClassFunctionAnalyzer.
    
    Returns:
    dict: The points-to set.
    """

### &#128161; In the following cell, you will test your new implementation and print the new points-to set.

In [ ]:



# Expected output:
# {
#     "obj": [
#         "French",
#         "English",
#         "German",
#         "Spanish"
#     ]
# }

Perfect! Now the points-to set of the `obj` object is refined!

‼️But! 
Can "obj" be a German or Spanish at line 38 (obj.say_hello("Patrick"))?
The answer is NO! because so far our analysis was not precise enough to discover that, it is more precise than the "hierarchy" technique but it is still too much of an approximation.

### &#128161;Flow-sensitive analysis

#### Your goal now is to give two different points-to set for the "obj" variable given their location in the code.

We will not implement here a full-fledged data flow analysis to propagate the points-to set by following the control flow, it is too advanced.
I would like you to refine the points-to set so far constructed and only consider class instantiation that only occured before the call on an object.

For instance, in our case, at line 38, the points-to set should be equal to ["English", "French"] and at line 45, it should be ["German", "Spanish", "English", "French"].

Your turn!

In [ ]:
def build_points_to_set(node, analyzer):
    """
    Analyzes an AST to gather information about potential class instantiations and method calls.
    
    Parameters:
    node: The root node of the AST.
    analyzer: An instance of ClassFunctionAnalyzer.
    
    Returns:
    dict: The points-to set.
    """

### &#128161; In the following cell, you will test your new implementation and print the new points-to set.

In [ ]:


# Expected output:
# {
#     "obj@37": [
#         "French",
#         "English"
#     ],
#     "obj@44": [
#         "French",
#         "English",
#         "German",
#         "Spanish"
#     ]
# }

&#128161; Now write a function to print the points-to set in a human-readable format, for instance:
Variable "obj" at line "XX" has the following points-to set: YY, ZZ

In [ ]:
def print_points_to_set(points_to_set):
    """
    Prints the points-to set in a readable format.

    Parameters:
    points_to_set: The points-to set.
    """

In [ ]:
print_points_to_set(points_to_set)

Congratulations! 

You have refined the points-to set. 
But, according to you, can it be refined further?
How? Explain. 